# Read Text In From File

In [33]:
# with open('/ds/hohsiangwu/rosetta/Python/train.function', 'r') as f:
#     t_code = f.readlines()

# with open('/ds/hohsiangwu/rosetta/Python/valid.function', 'r') as f:
#     v_code = f.readlines()

# with open('/ds/hohsiangwu/rosetta/Python/test.function', 'r') as f:
#     holdout_code = f.readlines()

# train_code = t_code + v_code

# print(f'train_code rows: {len(train_code):,}')
# print(f'holdout_code rows: {len(holdout_code):,}')
# print(f'total code rows: {len(holdout_code + train_code):,}')


# with open('/ds/hohsiangwu/rosetta/Python/train.docstring', 'r') as f:
#     t_comment = f.readlines()

# with open('/ds/hohsiangwu/rosetta/Python/valid.docstring', 'r') as f:
#     v_comment = f.readlines()

# with open('/ds/hohsiangwu/rosetta/Python/test.docstring', 'r') as f:
#     holdout_comment = f.readlines()

# train_comment = t_comment + v_comment

# print(f'\ntrain_comment rows: {len(train_comment):,}')
# print(f'holdout_comment rows: {len(holdout_comment):,}')
# print(f'total comment rows: {len(holdout_comment + train_comment):,}')

train_code rows: 1,426,330
holdout_code rows: 75,071
total code rows: 1,501,401

train_comment rows: 1,426,330
holdout_comment rows: 75,071
total comment rows: 1,501,401


# Tokenize Text

In [34]:
# from ktext.preprocess import processor
# code_proc = processor(hueristic_pct_padding=.7, keep_n=10000)
# t_code = code_proc.fit_transform(train_code)

# comment_proc = processor(append_indicators=True, hueristic_pct_padding=.7, keep_n=8000, padding ='post')
# t_comment = comment_proc.fit_transform(train_comment)

 See full histogram by insepecting the `document_length_stats` attribute.
 See full histogram by insepecting the `document_length_stats` attribute.


Save tokenized text

In [35]:
# import dill as dpickle
# import numpy as np

# # Save the preprocessor
# with open(PATH/'py_code_proc.dpkl', 'wb') as f:
#     dpickle.dump(code_proc, f)

# with open(PATH/'py_comment_proc.dpkl', 'wb') as f:
#     dpickle.dump(comment_proc, f)

# # Save the processed data
# np.save(PATH/'py_t_code_vecs.npy', t_code)
# np.save(PATH/'py_t_comment_vecs.npy', t_comment)

Arrange data for modeling

In [2]:
%reload_ext autoreload
%autoreload 2
from seq2seq_utils import load_decoder_inputs, load_encoder_inputs, load_text_processor

encoder_input_data, doc_length = load_encoder_inputs(PATH/'py_t_code_vecs.npy')
decoder_input_data, decoder_target_data = load_decoder_inputs(PATH/'py_t_comment_vecs.npy')

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Shape of encoder input: (4978625, 55)
Shape of decoder input: (4978625, 14)
Shape of decoder target: (4978625, 14)


In [3]:
num_encoder_tokens, body_pp = load_text_processor(PATH/'py_code_proc.dpkl')
num_decoder_tokens, title_pp = load_text_processor(PATH/'py_comment_proc.dpkl')

Size of vocabulary for /ds/hamel/CodeML/Get_Python_From_BigQuery/py_code_proc.dpkl: 20,002
Size of vocabulary for /ds/hamel/CodeML/Get_Python_From_BigQuery/py_comment_proc.dpkl: 15,002


# Build Seq2Seq Model For Summarizing Code - (This Is Only For Transfer Learning)

Will reuse this for the code search task

In [5]:
%matplotlib inline
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding, Bidirectional, BatchNormalization
from keras import optimizers

In [10]:
#arbitrarly set latent dimension for embedding and hidden units
latent_dim = 800

##### Define Model Architecture ######

########################
#### Encoder Model ####
encoder_inputs = Input(shape=(doc_length,), name='Encoder-Input')

# Word embeding for encoder (ex: Issue Body)
x = Embedding(num_encoder_tokens, latent_dim, name='Body-Word-Embedding', mask_zero=False)(encoder_inputs)
x = BatchNormalization(name='Encoder-Batchnorm-1')(x)

# Intermediate GRU layer (optional)
# x = GRU(latent_dim, name='Encoder-Intermediate-GRU', return_sequences=True)(x)
# x = BatchNormalization(name='Encoder-Batchnorm-2')(x)

# We do not need the `encoder_output` just the hidden state.
_, state_h = GRU(latent_dim, return_state=True, name='Encoder-Last-GRU', dropout=.5)(x)

# Encapsulate the encoder as a separate entity so we can just 
#  encode without decoding if we want to.
encoder_model = Model(inputs=encoder_inputs, outputs=state_h, name='Encoder-Model')

seq2seq_encoder_out = encoder_model(encoder_inputs)

########################
#### Decoder Model ####
decoder_inputs = Input(shape=(None,), name='Decoder-Input')  # for teacher forcing

# Word Embedding For Decoder (ex: Issue Titles)
dec_emb = Embedding(num_decoder_tokens, latent_dim, name='Decoder-Word-Embedding', mask_zero=False)(decoder_inputs)
dec_bn = BatchNormalization(name='Decoder-Batchnorm-1')(dec_emb)

# Set up the decoder, using `decoder_state_input` as initial state.
decoder_gru = GRU(latent_dim, return_state=True, return_sequences=True, name='Decoder-GRU', dropout=.5)
decoder_gru_output, _ = decoder_gru(dec_bn, initial_state=seq2seq_encoder_out)
x = BatchNormalization(name='Decoder-Batchnorm-2')(decoder_gru_output)

# Dense layer for prediction
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='Final-Output-Dense')
decoder_outputs = decoder_dense(x)

########################
#### Seq2Seq Model ####

#seq2seq_decoder_out = decoder_model([decoder_inputs, seq2seq_encoder_out])
seq2seq_Model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [11]:
seq2seq_Model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder-Input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Decoder-Word-Embedding (Embeddi (None, None, 800)    12001600    Decoder-Input[0][0]              
__________________________________________________________________________________________________
Encoder-Input (InputLayer)      (None, 55)           0                                            
__________________________________________________________________________________________________
Decoder-Batchnorm-1 (BatchNorma (None, None, 800)    3200        Decoder-Word-Embedding[0][0]     
__________________________________________________________________________________________________
Encoder-Mo

In [13]:
from keras.callbacks import CSVLogger, ModelCheckpoint
import numpy as np

seq2seq_Model.compile(optimizer=optimizers.Nadam(lr=0.0015), loss='sparse_categorical_crossentropy')

script_name_base = 'py_func_sum_v2_'
csv_logger = CSVLogger('{:}.log'.format(script_name_base))
model_checkpoint = ModelCheckpoint('{:}.epoch{{epoch:02d}}-val{{val_loss:.5f}}.hdf5'.format(script_name_base),
                                   save_best_only=True)

batch_size = 900
epochs = 60
history = seq2seq_Model.fit([encoder_input_data, decoder_input_data], np.expand_dims(decoder_target_data, -1),
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.12, callbacks=[csv_logger, model_checkpoint])

Train on 4381190 samples, validate on 597435 samples
Epoch 1/60
4381190/4381190 [==============================] - 1969s 449us/step - loss: 1.9850 - val_loss: 1.9785


/opt/conda/lib/python3.6/site-packages/keras/engine/topology.py:2379: UserWarning: Layer Decoder-GRU was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'Encoder-Model_2/Encoder-Last-GRU/while/Exit_2:0' shape=(?, 800) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


Epoch 2/60
4381190/4381190 [==============================] - 1976s 451us/step - loss: 1.9235 - val_loss: 1.9470
Epoch 3/60
4381190/4381190 [==============================] - 1983s 453us/step - loss: 1.8917 - val_loss: 1.9235
Epoch 4/60
4381190/4381190 [==============================] - 1979s 452us/step - loss: 1.8578 - val_loss: 1.9043
Epoch 5/60
4381190/4381190 [==============================] - 1985s 453us/step - loss: 1.8345 - val_loss: 1.8900
Epoch 6/60
4381190/4381190 [==============================] - 1983s 453us/step - loss: 1.8143 - val_loss: 1.8767
Epoch 7/60
4381190/4381190 [==============================] - 1993s 455us/step - loss: 1.7969 - val_loss: 1.8668
Epoch 8/60
4381190/4381190 [==============================] - 1996s 456us/step - loss: 1.7849 - val_loss: 1.8539
Epoch 9/60
4381190/4381190 [==============================] - 1984s 453us/step - loss: 1.7689 - val_loss: 1.8465
Epoch 10/60
4381190/4381190 [==============================] - 2008s 458us/step - loss: 1.7578 -

In [17]:
from pathlib import Path
PATH = Path('/ds/hamel/CodeML/Get_Python_From_BigQuery')
#seq2seq_Model.save(PATH/'seq2seq_code_search_py_v2.hdf5')

/opt/conda/lib/python3.6/site-packages/keras/engine/topology.py:2379: UserWarning: Layer Decoder-GRU was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'Encoder-Model_2/Encoder-Last-GRU/while/Exit_2:0' shape=(?, 800) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


# If notebook dies can restart from here

In [18]:
from keras.models import load_model
from seq2seq_utils import load_text_processor
from pathlib import Path
PATH = Path('/ds/hamel/CodeML/Get_Python_From_BigQuery')

num_encoder_tokens, body_pp = load_text_processor(PATH/'py_code_proc.dpkl')
num_decoder_tokens, title_pp = load_text_processor(PATH/'py_comment_proc.dpkl')
seq2seq_Model = load_model(PATH/'seq2seq_code_search_py_v2.hdf5')

Size of vocabulary for /ds/hamel/CodeML/Get_Python_From_BigQuery/py_code_proc.dpkl: 20,002
Size of vocabulary for /ds/hamel/CodeML/Get_Python_From_BigQuery/py_comment_proc.dpkl: 15,002


In [19]:
from seq2seq_utils import Seq2Seq_Inference
seq2seq_inf = Seq2Seq_Inference(encoder_preprocessor=body_pp,
                                 decoder_preprocessor=title_pp,
                                 seq2seq_model=seq2seq_Model)

In [20]:
import pandas as pd
PATH = Path('/ds/hohsiangwu/projects/semantic_search')

with open(PATH/'test.function', 'r') as f:
    holdout_code = f.readlines()

with open(PATH/'test.docstring', 'r') as f:
    holdout_comment = f.readlines()


demo_testdf = pd.DataFrame({'body':holdout_code, 'issue_title':holdout_comment, 'issue_url':''})
seq2seq_inf.demo_model_predictions(n=5, issue_df=demo_testdf)


from pathlib import Path
PATH = Path('/ds/hamel/CodeML/Get_Python_From_BigQuery')



============== Example # 15635 =================

Issue Body:
 app route login methods GET POST def login if g user return redirect url_for timeline error None if request method POST user query_db select from user where username request form username one True if user is None error Invalid username elif not check_password_hash user pw_hash request form password error Invalid password else flash You were logged in session user_id user user_id return redirect url_for timeline return render_template login html error error
 

Original Title:
 logs the user in .


****** Machine Generated Title (Prediction) ******:
 logs the user in


============== Example # 21181 =================

Issue Body:
 def make_user_from_json user_dict user User int user_dict age str user_dict sex user_dict hobbies user_dict userType user_dict lovedCategories user_dict alreadyGifted return user
 

Original Title:
 generate user : return : user


****** Machine Generated Title (Prediction) ******:
 makes a dictio

# Fine Tune This To Predict Universal Sentence Encoder Vector

In [14]:
import tensorflow as tf
import tensorflow_hub as hub
embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/1")

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_0:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with Embeddings_en/sharded_0


INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_0:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with Embeddings_en/sharded_0


INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_1:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with Embeddings_en/sharded_1


...

INFO:tensorflow:Initialize variable module/SNLI/Classifier/tanh_layer_0/bias:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with SNLI/Classifier/tanh_layer_0/bias


INFO:tensorflow:Initialize variable module/SNLI/Classifier/tanh_layer_0/weights:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with SNLI/Classifier/tanh_layer_0/weights


INFO:tensorflow:Initialize variable module/SNLI/Classifier/tanh_layer_0/weights:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with SNLI/Classifier/tanh_layer_0/weights


INFO:tensorflow:Initialize variable module/global_step:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with global_step


INFO:tensorflow:Initialize variable module/global_step:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with global_step


In [15]:
tf.__version__

'1.7.0'

In [16]:
with tf.Session() as session:
    def get_embeddings(text_blob_list):
            emb = session.run(embed(text_blob_list))
            return emb

In [8]:
test = get_embeddings(train_comment[:1000])
test.shape

(1000, 512)

In [9]:
from more_itertools import chunked
train_chunked = list(chunked(train_comment, 10000))

In [7]:
i = 1
use_emb_list = []

with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    for x in train_chunked:
        print(i)
        i+=1
        #75th percentile of comment length is 89 characters, so I made limit 200.
        use_emb_list.append(get_embeddings([s[:200] for s in x]))

In [17]:
# use_emb = np.concatenate(use_emb_list)

In [22]:
# save the embedding !
# np.save('use_emb.npy', use_emb)

# Train Mapping Function To Sentence Embedding Space

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import numpy as np
from seq2seq_utils import extract_encoder_model, load_encoder_inputs
from keras.layers import Input, Dense, BatchNormalization, Dropout, Lambda

from keras.models import load_model, Model
from seq2seq_utils import load_text_processor

from pathlib import Path
PATH = Path('/ds/hamel/CodeML/Get_Python_From_BigQuery')

num_encoder_tokens, body_pp = load_text_processor(PATH/'py_code_proc.dpkl')
num_decoder_tokens, title_pp = load_text_processor(PATH/'py_comment_proc.dpkl')
encoder_input_data, doc_length = load_encoder_inputs(PATH/'py_t_code_vecs.npy')
seq2seq_Model = load_model(PATH/'seq2seq_code_search_py_v2.hdf5')

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Size of vocabulary for /ds/hamel/CodeML/Get_Python_From_BigQuery/py_code_proc.dpkl: 20,002
Size of vocabulary for /ds/hamel/CodeML/Get_Python_From_BigQuery/py_comment_proc.dpkl: 15,002
Shape of encoder input: (4978625, 55)


In [2]:
# Load Fitlam Embeddings
fastailm_emb = np.load(PATH/'combined_fastailm_emb.npy')

In [3]:
fastailm_emb.shape

(4978625, 1200)

In [4]:
encoder_input_data.shape

(4978625, 55)

In [27]:
# Extract Encoder from seq2seq model
encoder_model = extract_encoder_model(seq2seq_Model)
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Encoder-Input (InputLayer)   (None, 55)                0         
_________________________________________________________________
Body-Word-Embedding (Embeddi (None, 55, 800)           16001600  
_________________________________________________________________
Encoder-Batchnorm-1 (BatchNo (None, 55, 800)           3200      
_________________________________________________________________
Encoder-Last-GRU (GRU)       [(None, 800), (None, 800) 3842400   
Total params: 19,847,200
Trainable params: 0
Non-trainable params: 19,847,200
_________________________________________________________________


In [28]:
# Freeze Encoder Model
for l in encoder_model.layers:
    l.trainable = False
    print(l, l.trainable)

<keras.engine.topology.InputLayer object at 0x7f40c798f9b0> False
<keras.layers.embeddings.Embedding object at 0x7f40c798fac8> False
<keras.layers.normalization.BatchNormalization object at 0x7f40c798fb70> False
<keras.layers.recurrent.GRU object at 0x7f40c798fcf8> False


### Construct New Model Architecture

In [31]:
#Figure out range of Sentence Encoder
fastailm_emb.max(), fastailm_emb.min(), fastailm_emb.mean()

(1.0, -0.9999999, 0.10509811)

In [33]:
########################
#### Encoder Model ####

encoder_inputs = Input(shape=(doc_length,), name='Encoder-Input')
enc_out = encoder_model(encoder_inputs)

# first dense layer with batch norm
fc = Dense(1024, activation='relu')(enc_out)
fc = BatchNormalization(name='bn-1')(fc)

# force network to bound output between (.152, -.152) to match target range
fc = Dense(1200, activation='tanh')(fc)
#target_emb = Lambda(lambda x: x * .152)(fc)

codeSearch_Model = Model([encoder_inputs], fc)

In [34]:
codeSearch_Model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Encoder-Input (InputLayer)   (None, 55)                0         
_________________________________________________________________
Encoder-Model (Model)        (None, 800)               19847200  
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              820224    
_________________________________________________________________
bn-1 (BatchNormalization)    (None, 1024)              4096      
_________________________________________________________________
dense_4 (Dense)              (None, 1200)              1230000   
Total params: 21,901,520
Trainable params: 2,052,272
Non-trainable params: 19,849,248
_________________________________________________________________


In [39]:
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras import optimizers

codeSearch_Model.compile(optimizer=optimizers.Nadam(lr=0.005), loss='cosine_proximity')
script_name_base = 'cs_model_fitlam_'
csv_logger = CSVLogger('{:}.log'.format(script_name_base))
model_checkpoint = ModelCheckpoint('{:}.epoch{{epoch:02d}}-val{{val_loss:.5f}}.hdf5'.format(script_name_base),
                                   save_best_only=True)

batch_size = 5000
epochs = 50
history = codeSearch_Model.fit([encoder_input_data], fastailm_emb,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.12, callbacks=[csv_logger, model_checkpoint])

Train on 4381190 samples, validate on 597435 samples
Epoch 1/50
4381190/4381190 [==============================] - 286s 65us/step - loss: -0.8031 - val_loss: -0.7955
Epoch 2/50
4381190/4381190 [==============================] - 291s 66us/step - loss: -0.8048 - val_loss: -0.8006
Epoch 3/50
4381190/4381190 [==============================] - 285s 65us/step - loss: -0.8050 - val_loss: -0.8014
Epoch 4/50
4381190/4381190 [==============================] - 291s 66us/step - loss: -0.8053 - val_loss: -0.8000
Epoch 5/50
4381190/4381190 [==============================] - 284s 65us/step - loss: -0.8055 - val_loss: -0.8028
Epoch 6/50
4381190/4381190 [==============================] - 291s 66us/step - loss: -0.8000 - val_loss: -0.8031
Epoch 7/50
4381190/4381190 [==============================] - 284s 65us/step - loss: -0.8059 - val_loss: -0.8037
Epoch 8/50
4381190/4381190 [==============================] - 291s 66us/step - loss: -0.8061 - val_loss: -0.8034
Epoch 9/50
4381190/4381190 [===============

In [40]:
#codeSearch_Model.save(PATH/'codeSearch_Model_frozen.hdf5')

In [10]:
PATH

PosixPath('/ds/hamel/CodeML/Get_Python_From_BigQuery')

In [6]:
codeSearch_Model = load_model(PATH/'codeSearch_Model_frozen.hdf5')

In [11]:
codeSearch_Model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Encoder-Input (InputLayer)   (None, 55)                0         
_________________________________________________________________
Encoder-Model (Model)        (None, 800)               19847200  
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              820224    
_________________________________________________________________
bn-1 (BatchNormalization)    (None, 1024)              4096      
_________________________________________________________________
dense_4 (Dense)              (None, 1200)              1230000   
Total params: 21,901,520
Trainable params: 2,052,272
Non-trainable params: 19,849,248
_________________________________________________________________


# Test Viability of Lookups In This Vectorspace

In [8]:
# Vectorize all the code!
batch_size = 1300
train_vecs = codeSearch_Model.predict(encoder_input_data, batch_size=batch_size)

In [9]:
import numpy as np
np.save(PATH/'train_code_to_lang_vecs.npy', train_vecs)

In [77]:
# load vectorized code
train_vecs = np.load(PATH/'train_code_to_lang_vecs.npy')
train_vecs.shape

((75071, 512), (1426330, 512))

In [90]:
def vec_eng_query(text):
    return get_embeddings([text])

In [94]:
import nmslib
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(train_vecs)
index.createIndex({'post': 2}, print_progress=True)

In [123]:
ref_code = t_code + v_code

In [124]:
test = vec_eng_query('Change colors on web page.')
ids, distances = index.knnQuery(test, k=1)

In [125]:
print(ref_code[ids[0]])

def set_bgcolor self bgcolor assert bgcolor startswith color gtk gdk color_parse bgcolor self modify_bg gtk STATE_NORMAL color



In [126]:
test = vec_eng_query('aggregate and sort data.')
ids, distances = index.knnQuery(test, k=1)

In [127]:
print(ref_code[ids[0]])

def stats self stats self _stats if stats is None db current db s3db current s3db utable s3db auth_user otable s3db org_organisation left otable on otable id utable organisation_id query utable deleted False users db query select otable root_organisation utable registration_key utable timestmp left left import datetime now current request utcnow start now datetime timedelta days 30 replace hour 0 minute 0 second 0 microsecond 0 stats for user in users account user auth_user organisation user org_organisation root_org organisation root_organisation if not root_org continue if root_org in stats org_stats stats root_org else org_stats stats root_org 0 0 0 org_stats 1 if account registration_key org_stats 1 timestmp account timestmp if timestmp and timestmp start org_stats 1 self _stats stats return stats



In [122]:
len(t_code)

1283697